#PROJECT

In [39]:
import json
import pandas as pd
import requests

# Effettua la richiesta GET all'API TfL
url = "https://api.tfl.gov.uk/AccidentStats/2019"  # Cambia l'anno a seconda dei dati che vuoi
response = requests.get(url)

# Carica la risposta in formato JSON
data = response.json()

# Converte il JSON in un DataFrame di pandas
df = pd.json_normalize(data)

# Salva il DataFrame in formato CSV
df.to_csv('incidenti_tfl_2019.csv', index=False)

In [40]:
df.head()

,$type,id,lat,lon,location,date,severity,borough,casualties,vehicles
0,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,345979,51.570865,-0.231959,On Edgware Road Near The Junction With north C...,2019-01-04T21:22:00Z,Slight,Barnet,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
1,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,345980,51.603859,-0.187240,On Willow Way Near The Junction With Long Lane,2019-01-04T23:33:00Z,Slight,Barnet,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
2,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,345981,51.512198,-0.153122,On north Audley Street Near The Junction With ...,2019-01-04T22:15:00Z,Slight,City of Westminster,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
3,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,345982,51.431480,-0.016083,On Bromley Road Near The Junction With Daneswo...,2019-01-04T18:00:00Z,Slight,Lewisham,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
4,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,345983,51.473487,0.145202,On Belmont Road Near The Junction With Bedonwe...,2019-01-04T20:45:00Z,Slight,Bexley,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...


In [41]:
df.describe()

,id,lat,lon
count,50626.000000,50626.000000,50626.000000
mean,544054.000000,51.509198,-0.118897
std,185715.647807,0.067383,0.141133
min,345828.000000,51.295233,-0.509795
25%,358484.250000,51.468225,-0.200805
50%,544054.000000,51.514006,-0.114336
75%,729623.750000,51.553555,-0.035484
max,742280.000000,51.691126,0.307909


In [42]:
df.drop(columns=['$type'], inplace=True)

# converting 'severity' in cardinal indicator
df['severity'] = df['severity'].map({'Slight': 1, 'Serious': 2, 'Fatal': 3})
df.head()

,id,lat,lon,location,date,severity,borough,casualties,vehicles
0,345979,51.570865,-0.231959,On Edgware Road Near The Junction With north C...,2019-01-04T21:22:00Z,1,Barnet,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
1,345980,51.603859,-0.187240,On Willow Way Near The Junction With Long Lane,2019-01-04T23:33:00Z,1,Barnet,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
2,345981,51.512198,-0.153122,On north Audley Street Near The Junction With ...,2019-01-04T22:15:00Z,1,City of Westminster,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
3,345982,51.431480,-0.016083,On Bromley Road Near The Junction With Daneswo...,2019-01-04T18:00:00Z,1,Lewisham,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
4,345983,51.473487,0.145202,On Belmont Road Near The Junction With Bedonwe...,2019-01-04T20:45:00Z,1,Bexley,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...


In [43]:
borough_encoded = pd.get_dummies(df['borough'], prefix='borough', dtype=int)
df = pd.concat([df, borough_encoded], axis=1)
df.drop('borough', axis=1, inplace=True)

df.head()

,id,lat,lon,location,date,severity,casualties,vehicles,borough_Barking and Dagenham,borough_Barnet,...,borough_Lewisham,borough_Merton,borough_Newham,borough_Redbridge,borough_Richmond upon Thames,borough_Southwark,borough_Sutton,borough_Tower Hamlets,borough_Waltham Forest,borough_Wandsworth
0,345979,51.570865,-0.231959,On Edgware Road Near The Junction With north C...,2019-01-04T21:22:00Z,1,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,0,1,...,0,0,0,0,0,0,0,0,0,0
1,345980,51.603859,-0.187240,On Willow Way Near The Junction With Long Lane,2019-01-04T23:33:00Z,1,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,0,1,...,0,0,0,0,0,0,0,0,0,0
2,345981,51.512198,-0.153122,On north Audley Street Near The Junction With ...,2019-01-04T22:15:00Z,1,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,0,0,...,0,0,0,0,0,0,0,0,0,0
3,345982,51.431480,-0.016083,On Bromley Road Near The Junction With Daneswo...,2019-01-04T18:00:00Z,1,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,0,0,...,1,0,0,0,0,0,0,0,0,0
4,345983,51.473487,0.145202,On Belmont Road Near The Junction With Bedonwe...,2019-01-04T20:45:00Z,1,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
df.drop('location', axis=1, inplace=True)

In [45]:
df.head()

,id,lat,lon,date,severity,casualties,vehicles,borough_Barking and Dagenham,borough_Barnet,borough_Bexley,...,borough_Lewisham,borough_Merton,borough_Newham,borough_Redbridge,borough_Richmond upon Thames,borough_Southwark,borough_Sutton,borough_Tower Hamlets,borough_Waltham Forest,borough_Wandsworth
0,345979,51.570865,-0.231959,2019-01-04T21:22:00Z,1,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,345980,51.603859,-0.187240,2019-01-04T23:33:00Z,1,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,345981,51.512198,-0.153122,2019-01-04T22:15:00Z,1,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,345982,51.431480,-0.016083,2019-01-04T18:00:00Z,1,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,345983,51.473487,0.145202,2019-01-04T20:45:00Z,1,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Now creating a separate dataframe for casualties and one for vehicles, in order to have more readable data.

In [46]:
df_casualties = df.copy()
df_casualties = df_casualties.explode('casualties')
df_casualties.drop(columns=['lat', 'lat', 'lon', 'date', 'location', 'severity', 'borough', 'vehicles'], inplace=True)
df_casualties.head()

KeyError: "['location', 'borough'] not found in axis"

In [ ]:
type(df_casualties['casualties'].values)
df_casualties_normalized = pd.json_normalize(df_casualties['casualties'].values)
df_casualties_normalized.drop(columns=['$type'], inplace=True)
df_casualties_normalized.head()


,age,class,severity,mode,ageBand
0,58.0,Driver,Slight,Car,Adult
1,NaN,Passenger,Slight,Car,Unknown
2,NaN,Passenger,Slight,Car,Unknown
3,24.0,Driver,Slight,Car,Adult
4,21.0,Passenger,Slight,PedalCycle,Adult


In [ ]:
df_casualties_normalized['mode'].unique()

array(['Car', 'PedalCycle', 'Pedestrian', 'Taxi', 'PoweredTwoWheeler',
       'BusOrCoach', 'PrivateHire', 'OtherVehicle', 'GoodsVehicle'],
      dtype=object)

In [ ]:
df_casualties = df_casualties.join(df_casualties_normalized)
df_casualties.drop(columns=['casualties'], inplace=True)
df_casualties.head()

,id,age,class,severity,mode,ageBand
0,345828,58.0,Driver,Slight,Car,Adult
0,345828,58.0,Driver,Slight,Car,Adult
0,345828,58.0,Driver,Slight,Car,Adult
1,345829,NaN,Passenger,Slight,Car,Unknown
2,345830,NaN,Passenger,Slight,Car,Unknown


In [ ]:
df_vehicles = df.copy()
df_vehicles = df_vehicles.explode('vehicles')
df_vehicles.drop(columns=['lat', 'lat', 'lon', 'date', 'severity', 'borough', 'casualties'], inplace=True)

df_vehicles_normalized = pd.json_normalize(df_vehicles['vehicles'].values)
df_vehicles_normalized.drop(columns=['$type'], inplace=True)

df_vehicles = df_vehicles.join(df_vehicles_normalized)
df_vehicles.drop(columns=['vehicles'], inplace=True)
df_vehicles.head()

KeyError: "['location', 'borough'] not found in axis"

In [ ]:
df_vehicles['type'].unique()

TypeError: unhashable type: 'list'

In [ ]:
# collapse row with same id, add column with number of vehicles, add column with list of vehicles types
# it also sorts indexes idk why
df_vehicles = df_vehicles.groupby(['id']).agg({'type': lambda x: list(x)}).reset_index()
df_vehicles['num_vehicles'] = df_vehicles['type'].apply(lambda x: len(x))
df_vehicles

,id,type,num_vehicles
0,345828,"[Car, Car]",2
1,345829,"[Car, Car]",2
2,345830,"[Car, Car]",2
3,345831,[Car],1
4,345832,"[PedalCycle, PedalCycle]",2
...,...,...,...
50621,742276,[PedalCycle],1
50622,742277,[Car],1
50623,742278,"[PedalCycle, PedalCycle]",2
50624,742279,"[Taxi, Taxi]",2


# NOTES SECTION

## 3 OPTIONS:

* Forcasting
 * Determine out of the dataset what would happen
 * Predict accidents most likely
 * we could use past year data and compare directly with more recent data

* Corelating variables
 * Finding relationship betweetn different variables
 * expanding knowledge base to variables such as
 * Street direction, speed limit, street light, roundabout presence
 * tackle problems that normal statistics patterns or resarch can struggle with
 * Anomaly detection


* Optimization
 * Optimize the dataset
 * Most likely determining the best way to have the London transportation system
 * A simulation, hard to determine results




We could study what subjects are more at risk of severe accidents

What parts of the city are the most at risk

For insurance purposes


IDEA:
We could study the correlation between variables and accidents -> building a cost function to determine the reliability of a certain subject
Could be useful for INSURANCE COMPANIES

hi